In [1]:
!pip install pandas pyarrow
!pip install transformers datasets torch pandas numpy


In [2]:
!pip install transformers datasets torch pandas numpy


In [3]:
import pandas as pd

# Load the data (adjust paths based on your environment)
train_df = pd.read_parquet('/kaggle/input/unlp-2025-shared-task-span-identification/train.parquet')
test_df = pd.read_csv('/kaggle/input/unlp-2025-shared-task-span-identification/test.csv')

# Check the data
print("TRAIN DATA",train_df.head(2))
print("TEST DATA", test_df.head(2))

TRAIN DATA                                      id  \
0  0bb0c7fa-101b-4583-a5f9-9d503339141c   
1  7159f802-6f99-4e9d-97bd-6f565a4a0fae   

                                             content lang  manipulative  \
0  Новий огляд мапи DeepState від російського вій...   uk          True   
1  Недавно 95 квартал жёстко поглумился над русск...   ru          True   

                          techniques  \
0        [euphoria, loaded_language]   
1  [loaded_language, cherry_picking]   

                                   trigger_words  
0    [[27, 63], [65, 88], [90, 183], [186, 308]]  
1  [[0, 40], [123, 137], [180, 251], [253, 274]]  
TEST DATA                                      id  \
0  521cd2e8-dd9f-42c4-98ba-c0c8890ff1ba   
1  9b2a61e4-d14e-4ff7-b304-e73d720319bf   

                                             content  
0  Они просрали нашу технику, положили кучу людей...  
1  ❗️\nКитай предлагает отдать оккупированные тер...  


In [4]:
!pip install spacy

In [5]:
import re
import pandas as pd
import numpy as np
import spacy

# Load spaCy for word tokenization (generic multi-language tokenizer)
nlp = spacy.blank("xx")  # 'xx' is the generic tokenizer spaCy provides

# Function to clean text
def clean_text(text):
    # Lowercasing
    text = text.lower()
    # Remove URLs, mentions, hashtags, and special characters
    text = re.sub(r'http\S+|www\S+|#\S+|@\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Tokenizer function using spaCy
def tokenize_words(text):
    doc = nlp(text)  # Tokenize the text using spaCy's default tokenizer
    return [token.text for token in doc]

# Optimized function to parse spans directly from the list of lists (trigger_words)
def parse_spans(span_str):
    # Ensure the input is a string or handle other cases
    if isinstance(span_str, str):  # If it is a string
        if pd.isna(span_str) or span_str.strip() == "":  # Handle empty or NaN values
            return []
        try:
            # Extract all numbers from the string
            numbers = list(map(int, re.findall(r'\d+', span_str)))
            # Convert list into tuples of (start, end) spans
            if len(numbers) % 2 != 0:
                # If the number of extracted numbers is odd, it means one of the spans is incomplete
                return []
            spans = [(numbers[i], numbers[i + 1]) for i in range(0, len(numbers), 2)]
            return spans
        except Exception as e:
            # Log parsing errors for debugging
            print(f"Parsing error: {e} in {span_str}")
            return []
    elif isinstance(span_str, np.ndarray):  # If it is a numpy array, convert it to string and parse again
        span_str = ' '.join(str(x) for x in span_str)
        return parse_spans(span_str)  # Recurse with the converted string
    elif span_str is None:  # Handle NoneType
        return []
    else:
        # Handle cases where the input is not a string or numpy array
        print(f"Unexpected type: {type(span_str)}. Skipping this entry.")
        return []

# Load the data (adjust paths based on your environment)
train_df = pd.read_parquet('/kaggle/input/unlp-2025-shared-task-span-identification/train.parquet')
test_df = pd.read_csv('/kaggle/input/unlp-2025-shared-task-span-identification/test.csv')

# Apply text cleaning to both train and test data
train_df['cleaned_content'] = train_df['content'].apply(clean_text)
test_df['cleaned_content'] = test_df['content'].apply(clean_text)

# Tokenize train and test data by words
train_df['tokens'] = train_df['cleaned_content'].apply(tokenize_words)
test_df['tokens'] = test_df['cleaned_content'].apply(tokenize_words)

# Apply the function to the 'trigger_words' column of the train dataset to parse spans
train_df["parsed_spans"] = train_df["trigger_words"].apply(parse_spans)

# Display the first few rows to ensure correct processing
print(train_df[["trigger_words", "parsed_spans"]].head())

# Optionally, you can check the cleaned content and tokens as well
print("Sample cleaned content and tokens:")
print(train_df[['cleaned_content', 'tokens']].head())



                                   trigger_words  \
0    [[27, 63], [65, 88], [90, 183], [186, 308]]   
1  [[0, 40], [123, 137], [180, 251], [253, 274]]   
2                                    [[55, 100]]   
3                                           None   
4                                   [[114, 144]]   

                                    parsed_spans  
0    [(27, 63), (65, 88), (90, 183), (186, 308)]  
1  [(0, 40), (123, 137), (180, 251), (253, 274)]  
2                                    [(55, 100)]  
3                                             []  
4                                   [(114, 144)]  
Sample cleaned content and tokens:
                                     cleaned_content  \
0  новий огляд мапи deepstate від російського вій...   
1  недавно 95 квартал жёстко поглумился над русск...   
2  тим часом йде евакуація бєлгородського автовок...   
3  в україні найближчим часом мають намір посилит...   
4  расчёты 122мм сау 2с1 гвоздика 132й бригады 1г...   

         

In [6]:
# from transformers import AutoTokenizer
# import pandas as pd
# import numpy as np
# import re

# # Load the XLM-RoBERTa tokenizer
# model_name = "bert-base-multilingual-cased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Function to convert text and spans into tokens and labels
# def create_bio_labels(text, spans):
#     # Tokenize the text by splitting into words (not using tokenizer here, just word split)
#     tokens = text.split()  # Split by whitespace
#     labels = ["O"] * len(tokens)  # Initialize all labels as "O"
    
#     # Convert the start and end spans into token-based indices
#     for start_idx, end_idx in spans:
#         found = False
#         for i, token in enumerate(tokens):
#             token_start = sum(len(tokens[j]) + 1 for j in range(i))  # Count the length of the words before it
#             token_end = token_start + len(token) - 1

#             # Check if the token is within the span range
#             if token_start >= start_idx and token_end <= end_idx:
#                 if not found:  # First token of the span
#                     labels[i] = "B-MANIPULATIVE"
#                     found = True
#                 else:  # Inside the span
#                     labels[i] = "I-MANIPULATIVE"

#     # Ensure that the number of tokens matches the length of labels
#     assert len(tokens) == len(labels), f"Mismatch in tokens and labels lengths: {len(tokens)} vs {len(labels)}"
    
#     return tokens, labels

# # Example: Apply BIO tagging to the first few examples
# for i in range(2):
#     text = train_df.iloc[i]["cleaned_content"]  # Use the cleaned content
#     spans = train_df.iloc[i]["parsed_spans"]  # Use the parsed spans
#     tokens, labels = create_bio_labels(text, spans)

#     print(f"Example {i+1}:")
#     print("Tokens:", tokens)
#     print("Labels:", labels)
#     print("-" * 80)


In [7]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import re

# Load the XLM-RoBERTa tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to convert text and spans into tokens and labels
def create_bio_labels(text, spans):
    # Tokenize the text
    encoding = tokenizer(text, truncation=True, padding="max_length", max_length=512, return_offsets_mapping=True)
    
    # Convert input_ids to tokens (tokens are not directly available, so we convert from IDs)
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
    token_offsets = encoding['offset_mapping']
    
    # Initialize all labels as "O"
    labels = ["O"] * len(tokens)
    
    for start_idx, end_idx in spans:
        found = False
        for i, (token_start, token_end) in enumerate(token_offsets):
            # Check if the token is within the span range
            if token_start >= start_idx and token_end <= end_idx:
                if not found:  # First token of the span
                    labels[i] = "B-MANIPULATIVE"
                    found = True
                else:  # Inside the span
                    labels[i] = "I-MANIPULATIVE"
    
    # Ensure that the number of tokens matches the length of labels
    assert len(tokens) == len(labels), f"Mismatch in tokens and labels lengths: {len(tokens)} vs {len(labels)}"
    
    return tokens, labels

# Example: Apply BIO tagging to the first few examples
for i in range(2):
    text = train_df.iloc[i]["cleaned_content"]  # Use the cleaned content
    spans = train_df.iloc[i]["parsed_spans"]  # Use the parsed spans
    tokens, labels = create_bio_labels(text, spans)

    print(f"Example {i+1}:")
    print("Tokens:", tokens)
    print("Labels:", labels)
    print("-" * 80)


Example 1:
Tokens: ['[CLS]', 'новий', 'о', '##гляд', 'ма', '##пи', 'deep', '##state', 'від', 'рос', '##ійського', 'військового', 'е', '##кс', '##пер', '##та', 'к', '##уха', '##ра', 'пут', '##іна', '2', 'р', '##оз', '##ряду', 'с', '##пе', '##ці', '##алі', '##ста', 'по', 'с', '##нар', '##яд', '##ному', 'голо', '##ду', 'та', 'рек', '##тора', 'му', '##зи', '##чної', 'академії', 'м', '##ін', '##об', '##оро', '##ни', 'р', '##ф', 'є', '##в', '##г', '##є', '##нія', 'при', '##го', '##жина', 'при', '##го', '##жин', 'про', '##г', '##но', '##зу', '##є', 'що', 'не', '##в', '##дов', '##зі', 'нас', '##тан', '##е', 'день', 'зв', '##іль', '##нення', 'к', '##рим', '##у', 'і', 'день', 'р', '##оз', '##паду', 'рос', '##ії', 'каже', 'що', 'перед', '##ум', '##ови', 'цього', 'вже', 'ст', '##вор', '##ені', 'від', '##ео', 'взяли', 'з', 'канал', '##у', 'ф', '##д', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [8]:
from transformers import AutoTokenizer


In [ ]:
import time
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
from transformers import EarlyStoppingCallback
import torch.nn as nn
from transformers import DataCollatorForTokenClassification

# ✅ Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# Load XLM-Roberta Tokenizer
model_name = "xlm-roberta-large"  # Replacing XLM-RoBERTa with BERT Multilingual
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Label mapping
label2id = {"O": 0, "B-MANIPULATIVE": 1, "I-MANIPULATIVE": 2}
id2label = {v: k for k, v in label2id.items()}

# Function to encode BIO labels into numerical format
def encode_labels(labels):
    return [label2id[label] for label in labels]

# Process Train Data (you should have your train_df loaded here)
# Process Train Data
train_texts, train_labels_encoded = [], []
for _, row in train_df.iterrows():
    text = row["content"]
    spans = row["parsed_spans"]
    tokens, labels = create_bio_labels(text, spans)  # This will return both tokens and labels
    train_texts.append(text)
    train_labels_encoded.append(encode_labels(labels))


# ✅ Split dataset: 80% train, 20% validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels_encoded, test_size=0.2, random_state=42
)

# PyTorch Dataset Class
class SocialMediaDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(self.texts[idx], padding="max_length", truncation=True, max_length=128, return_tensors="pt")  # Reduced max_length
        label_ids = [-100] * len(encoding["input_ids"][0])  # Ignore padding tokens

        for i, label in enumerate(self.labels[idx][:len(label_ids)]):
            label_ids[i] = label

        encoding["labels"] = torch.tensor(label_ids)
        
        # Move tensors to the correct device
        return {key: val.squeeze(0).to(device) for key, val in encoding.items()}

# ✅ Create DataLoaders with reduced batch size
train_dataset = SocialMediaDataset(train_texts, train_labels)
val_dataset = SocialMediaDataset(val_texts, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)  # Reduced batch size to fit memory
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)  # Reduced batch size for validation

# BiLSTM-XLMRoBerta model
class BiLSTM_XLMRoBerta(nn.Module):
    def __init__(self, model_name, num_labels, hidden_size=1024, lstm_hidden_size=256, num_layers=2, dropout=0.1):
        super(BiLSTM_XLMRoBerta, self).__init__()
        
        # Load the pre-trained XLM-Roberta model
        self.transformer = AutoModelForTokenClassification.from_pretrained(
            model_name, 
            num_labels=num_labels
        ).base_model  # Extract base model without the classification head
        
        # BiLSTM Layer
        self.bilstm = nn.LSTM(
            input_size=hidden_size,  # Set to 1024 as XLM-Roberta's output size
            hidden_size=lstm_hidden_size, 
            num_layers=num_layers, 
            bidirectional=True, 
            batch_first=True, 
            dropout=dropout
        )
        
        # Linear layer for token classification
        self.classifier = nn.Linear(lstm_hidden_size * 2, num_labels)  # *2 because it's bidirectional

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Transformer output
        transformer_output = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = transformer_output.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)
        
        # BiLSTM output
        lstm_output, _ = self.bilstm(sequence_output)
        
        # Classifier output (logits)
        logits = self.classifier(lstm_output)  # Shape: (batch_size, seq_len, num_labels)

        # If labels are provided, calculate loss
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            # Flatten the tokens and labels for loss computation
            loss = loss_fct(logits.view(-1, self.classifier.out_features), labels.view(-1))

        return (loss, logits) if loss is not None else logits


# ✅ Load the new custom model
model = BiLSTM_XLMRoBerta(
    model_name="xlm-roberta-large", 
    num_labels=len(label2id),
    hidden_size=1024,  # XLM-Roberta output hidden size
    lstm_hidden_size=256,  # LSTM hidden size, you can tune this
    num_layers=2,  # Number of LSTM layers
    dropout=0.1  # Dropout for regularization
).to(device)

# ✅ Define Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# ✅ Training Arguments with gradient accumulation and FP16
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # ✅ Evaluates at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Reduced batch size
    num_train_epochs=27,  # More epochs
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,  # Gradient Accumulation for larger batch simulation
    fp16=True,  # Mixed precision for memory efficiency
)

# ✅ Early stopping callback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)  # Stops after 2 epochs of no improvement

# ✅ Trainer Class
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
    callbacks=[early_stopping_callback],  # Added early stopping
)

# ✅ Training Loop with Progress Bar & Metrics
print("🚀 Starting Training...")
start_time = time.time()

for epoch in range(training_args.num_train_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")

    for batch in progress_bar:
        optimizer.zero_grad()  # ✅ Clear previous gradients
        outputs = model(**batch)
        loss = outputs[0]  # Unpack loss from the tuple
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs[1], dim=2)  # Get logits from the second element of the tuple

        # ✅ Calculate Accuracy and F1 Score
        for i in range(len(batch["labels"])):
            true_labels = batch["labels"][i].cpu().numpy()
            pred_labels = preds[i].cpu().numpy()
            mask = true_labels != -100  # ✅ Ignore padding tokens
            correct += (true_labels[mask] == pred_labels[mask]).sum()
            total += mask.sum()

        avg_loss = total_loss / len(train_dataloader)
        accuracy = correct / total if total > 0 else 0

        # Compute F1 Score after each epoch
        f1 = f1_score(true_labels, pred_labels, average="weighted")  # F1 score for each batch

        progress_bar.set_postfix({"Loss": avg_loss, "Accuracy": accuracy, "F1": f1})

# ✅ Save Model
model.save_pretrained("xlm-roberta-large")  # Save the fine-tuned model
tokenizer.save_pretrained("xlm-roberta-large")

end_time = time.time()
print(f"✅ Training Completed in {round(end_time - start_time, 2)} seconds")
print(f"✅ Final Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}, F1: {f1:.4f}")

✅ Using device: cuda


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-d7e02277fda7>:151: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instea

🚀 Starting Training...


Epoch 5:  64%|██████▍   | 492/765 [03:58<02:12,  2.06it/s, Loss=0.398, Accuracy=0.743, F1=0.000121]